In [1]:
! pip install datasets transformers rouge-score nltk
!apt install git-lfs
!pip install sentencepiece
import transformers

import nltk
nltk.download('punkt')
import numpy as np

print(transformers.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 20.6 MB/s 
     |████████████████████████████████| 5.8 MB 21.0 MB/s 
     |████████████████████████████████| 182 kB 29.7 MB/s 
     |████████████████████████████████| 212 kB 38.5 MB/s 
     |████████████████████████████████| 115 kB 55.8 MB/s 
     |████████████████████████████████| 127 kB 12.5 MB/s 
     |████████████████████████████████| 7.6 MB 61.7 MB/s 
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=b2b641cccded4962287fed456248697f402ba588217dfb39d17c97405a23f70e
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Reading package lists... Done
Building

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


4.25.1


In [2]:
model_checkpoint = "t5-small"

from transformers import AutoTokenizer
import torch
from datasets import load_metric
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

metric = load_metric("rouge")

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:165: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingf

In [8]:
from datasets import load_dataset, load_metric, DatasetDict

#raw_datasets = load_dataset("xsum")
raw_datasets = load_dataset('cnn_dailymail',  '3.0.0')
#train_ds, test_ds = load_dataset('xsum', split=['train[0:1000]', 'test[0:100]'],revision="master")
#raw_datasets = load_dataset('xsum', split='train[:10%]+train[-80%:]')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  import sys


In [9]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [4]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [5]:
tokenized_datasets_original = raw_datasets.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets_original

NameError: ignored

In [12]:
import pickle
from google.colab import files

raw_datasets.save_to_disk("/content/drive/MyDrive/WikiFlow")
tokenized_datasets.save_to_disk("/content/drive/MyDrive/WikiFlow")

In [5]:
from datasets import load_from_disk
tokenized_datasets = load_from_disk("/content/drive/MyDrive/WikiFlow")
tokenized_datasets



DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})

In [6]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=0).select(range(20000))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=0).select(range(2500))

tokenized_datasets["train"] = small_train_dataset
tokenized_datasets["validation"] = small_eval_dataset
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})

In [7]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
print(device)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

cuda


In [8]:
batch_size = 32
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-temp3-CNNData",
    learning_rate=1e-2,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.001,
    save_total_limit=5,
    num_train_epochs=6,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=50,
    load_best_model_at_end=True
    #output_dir = "/content/drive/MyDrive/WikiFlow/Test Temp"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

from datasets import load_metric
metric = load_metric("rouge")

Using cuda_amp half precision backend


In [12]:
trainer.optim 

AttributeError: ignored

In [14]:
import torch
torch.cuda.empty_cache()

In [15]:
train_results = trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: highlights, id, article. If highlights, id, article are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 20000
  Num Epochs = 6
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3750
  Number of trainable parameters = 60506624


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
100,2.955300,2.534586,21.949500,9.012400,17.790400,20.580500,18.997200
200,2.858200,2.429757,21.804100,9.070100,17.840900,20.597000,19.000000
300,2.765500,2.404795,21.988900,9.354400,17.943100,20.768300,18.999600
400,2.738600,2.413934,22.351400,9.910900,18.446300,21.073000,19.000000
500,2.713900,2.352780,21.875600,9.369100,17.892100,20.659700,19.000000
600,2.662200,2.331400,22.537600,9.771100,18.462700,21.253500,18.996400
700,2.451200,2.333012,22.043000,9.668400,18.226800,20.846800,18.995600
800,2.491900,2.312102,22.698300,10.064900,18.736900,21.426900,19.000000
900,2.450900,2.290069,22.594000,9.909000,18.532300,21.384300,18.994000
1000,2.456400,2.286631,22.725300,9.728100,18.495300,21.351300,19.000000


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: highlights, id, article. If highlights, id, article are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2500
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: highlights, id, article. If highlights, id, article are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2500
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: highlights, id, article. If highlights, id, article are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore th

In [20]:
train_history = trainer.state.log_history
len(train_history)

113

In [18]:
### save model and trainer object
import pickle
with open('/content/drive/MyDrive/WikiFlow/Test3/train_history.pickle', 'wb') as handle:
    pickle.dump(train_history, handle, protocol=pickle.HIGHEST_PROTOCOL)
trainer.save_model("/content/drive/MyDrive/WikiFlow/Test3")

Saving model checkpoint to /content/drive/MyDrive/WikiFlow/Test3
Configuration saved in /content/drive/MyDrive/WikiFlow/Test3/config.json
Model weights saved in /content/drive/MyDrive/WikiFlow/Test3/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/WikiFlow/Test3/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/WikiFlow/Test3/special_tokens_map.json
Copy vocab file to /content/drive/MyDrive/WikiFlow/Test3/spiece.model


In [1]:
### load model
modelTemp = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/WikiFlow/Test3")
#modelTemp.load_state_dict(torch.load("/content/drive/MyDrive/WikiFlow/Test2"))

'''
with open('trainer.pickle', 'rb') as handle:
    trainHist = pickle.load(handle)
'''

NameError: ignored

In [24]:
modelTemp

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [9]:
example = "summarize: "+ tokenized_datasets['validation'][1]['article']
example
#tokenized_datasets['validation'][0]['article']

'summarize: (CNN)On the 6th of April 1996, San Jose Clash and DC United strode out in front of 31,683 expectant fans at the Spartan Stadium in San Jose, California. The historic occasion was the first ever Major League Soccer match -- a brave new dawn for the world\'s favorite sport in a land its charms had yet to conquer. Summarizing the action for ESPN, commentator Ty Keough eagerly described the momentous "birth of a new era for American soccer." Looking back at footage from that balmy evening now it\'s hard not to feel a certain nostalgia. Baggy shirts, questionable hairstyles and strange rule adaptations to make games more exciting were all part of the formative MLS experience. Countdown clocks were employed to provide drama at the end of each half. Even more bizarrely, tied games were settled by shootouts that saw attacking players run with the ball from 35-yards out before attempting to beat the opposing goalkeeper. As the MLS prepares to mark the beginning of its 20th season, i

In [10]:
tokenized_text = tokenizer.encode(example, return_tensors="pt").to(device)
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=128,
                                    early_stopping=True)
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(output)

Token indices sequence length is longer than the specified maximum sequence length for this model (1750 > 512). Running this sequence through the model will result in indexing errors


A new MLS deal with Orlando City is set for 2020. The new season is the first of a new domestic TV and media rights deal in the U.S. This year's season has been dubbed "the industry and the game itself has moved on dramatically" In addition to this, players will be able to compete with players with European passports, he says if they move on from the league, they can't afford to play in Europe, according to the report by the Kansas City Star newspaper. CLICK HERE for all the latest Real Madrid news,
